# Install and Load Libraries

In [1]:
import pandas as pd
import os
import pyarrow.parquet as pq
from glob import glob

# Joining CSV and Parquet Files

In [12]:
# Paths to data folders
train_csv_path = './data/train.csv'
test_csv_path = './data/test.csv'
series_train_path = './data/series_train.parquet'
series_test_path = './data/series_test.parquet'

# Load the CSV files
train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)

# Aggregation function for actigraphy data
def aggregate_actigraphy_data(series_path):
    participant_data = []

    for participant_folder in glob(os.path.join(series_path, 'id=*')):
        participant_id = os.path.basename(participant_folder).split('=')[1]
        parquet_file_path = os.path.join(participant_folder, 'part-0.parquet')
        
        if os.path.isfile(parquet_file_path):
            # Load the parquet file
            df = pq.read_table(parquet_file_path).to_pandas()
            
            # Aggregate metrics for each column
            agg_data = {
                'id': participant_id,
                
                # X, Y, Z columns: mean, std, min, max
                'mean_x': df['X'].mean(),
                'std_x': df['X'].std(),
                'min_x': df['X'].min(),
                'max_x': df['X'].max(),
                
                'mean_y': df['Y'].mean(),
                'std_y': df['Y'].std(),
                'min_y': df['Y'].min(),
                'max_y': df['Y'].max(),
                
                'mean_z': df['Z'].mean(),
                'std_z': df['Z'].std(),
                'min_z': df['Z'].min(),
                'max_z': df['Z'].max(),
                
                # ENMO and Angle-Z columns: mean, std, min, max
                'mean_enmo': df['enmo'].mean(),
                'std_enmo': df['enmo'].std(),
                'min_enmo': df['enmo'].min(),
                'max_enmo': df['enmo'].max(),
                
                'mean_anglez': df['anglez'].mean(),
                'std_anglez': df['anglez'].std(),
                'min_anglez': df['anglez'].min(),
                'max_anglez': df['anglez'].max(),
                
                # Non-wear flag: ratio of non-wear time
                'non_wear_ratio': df['non-wear_flag'].mean(),
                
                # Light and Battery Voltage columns: mean, std, min, max
                'mean_light': df['light'].mean(),
                'std_light': df['light'].std(),
                'min_light': df['light'].min(),
                'max_light': df['light'].max(),
                
                'mean_battery_voltage': df['battery_voltage'].mean(),
                'std_battery_voltage': df['battery_voltage'].std(),
                'min_battery_voltage': df['battery_voltage'].min(),
                'max_battery_voltage': df['battery_voltage'].max(),
                
                # Time of Day, Weekday, Quarter: mode (most frequent)
                'mode_time_of_day': df['time_of_day'].mode()[0] if not df['time_of_day'].mode().empty else None,
                'mode_weekday': df['weekday'].mode()[0] if not df['weekday'].mode().empty else None,
                'mode_quarter': df['quarter'].mode()[0] if not df['quarter'].mode().empty else None,
                
                # Relative date to PCIAT: mean, min, max
                'mean_relative_date_PCIAT': df['relative_date_PCIAT'].mean(),
                'min_relative_date_PCIAT': df['relative_date_PCIAT'].min(),
                'max_relative_date_PCIAT': df['relative_date_PCIAT'].max()
            }
            
            # Append the aggregated data for this participant
            participant_data.append(agg_data)
    
    # Convert list of dicts to DataFrame
    return pd.DataFrame(participant_data)

# Aggregate actigraphy data for train and test sets
aggregated_train_actigraphy = aggregate_actigraphy_data(series_train_path)
aggregated_test_actigraphy = aggregate_actigraphy_data(series_test_path)

# Merge aggregated actigraphy data with CSV data
merged_train_df = pd.merge(train_df, aggregated_train_actigraphy, on='id', how='left')
merged_test_df = pd.merge(test_df, aggregated_test_actigraphy, on='id', how='left')

# Export to CSV
#merged_train_df.to_csv('./data/merged_train.csv',index=False)
#merged_test_df.to_csv('./data/merged_test.csv',index=False)

In [ ]:
# Read in merged data
merged_train_df = pd.read_csv('./data/merged_train.csv')
merged_test_df = pd.read_csv('./data/merged_test.csv')